In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('{:.1f}GB RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('고성능 램 사용 안하고 있네')
else:
  print('고성능 램 사용중ㅋ!')

89.6GB RAM

고성능 램 사용중ㅋ!


# package install

In [ ]:
!pip install catboost==1.2.8
!pip install optuna==4.4.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 23.5 MB/s eta 0:00:00


# package load

In [ ]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import optuna
import os

# data load , seed

In [ ]:
test = pd.read_parquet('test.parquet')
train = pd.read_parquet('train.parquet')

seed = 14

# target data(Segment) C,D,E 모델링

In [ ]:
ab_ids = train[train['Segment'].isin(['A', 'B'])]['ID'].unique()

train = train[~train['ID'].isin(ab_ids)].copy()

In [ ]:
train['Segment'].value_counts()

,count
Segment,
E,1922052
D,349242
C,127590


In [ ]:
label_encoder = LabelEncoder()
train['Segment'] = label_encoder.fit_transform(train['Segment'])

X = train.drop(columns=['Segment', 'ID'])
y = train['Segment']
X_test = test.drop(columns=['ID'])

cat_features = [col for col in X.columns if X[col].dtype == 'object']
for col in cat_features:
    X[col] = X[col].astype(str)
    X_test[col] = X_test[col].astype(str)

In [ ]:
def objective(trial):
    bootstrap_type = trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli'])

    params = {
        'iterations': 1500,
        'learning_rate': trial.suggest_float("learning_rate", 0.01, 0.3),
        'depth': 8,
        'l2_leaf_reg': trial.suggest_float("l2_leaf_reg", 1.0, 10.0),
        'random_strength': trial.suggest_float("random_strength", 1e-9, 10.0),
        'border_count': trial.suggest_int("border_count", 32, 255),
        'bootstrap_type': bootstrap_type,
        'task_type': 'GPU',
        'loss_function': 'MultiClass',
        'eval_metric': 'Accuracy',
        'verbose': 0,
        'random_seed': seed,
        'class_weights': [2, 1, 1]
    }

    if bootstrap_type == 'Bayesian':
        params['bagging_temperature'] = trial.suggest_float("bagging_temperature", 0.0, 1.0)
    else:
        params['subsample'] = trial.suggest_float("subsample", 0.5, 1.0)

    X_train_sub, X_valid, y_train_sub, y_valid = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=seed
    )

    model = CatBoostClassifier(**params)
    model.fit(
        X_train_sub, y_train_sub,
        eval_set=(X_valid, y_valid),
        cat_features=cat_features,
        use_best_model=True,
        early_stopping_rounds=100
    )

    preds = model.predict(X_valid)
    score = accuracy_score(y_valid, preds)
    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=5)

best_params = study.best_trial.params

if best_params['bootstrap_type'] == 'Bayesian':
    best_params['bagging_temperature'] = study.best_trial.params['bagging_temperature']
else:
    best_params['subsample'] = study.best_trial.params['subsample']

best_params.update({
    'iterations': 1500,
    'loss_function': 'MultiClass',
    'eval_metric': 'Accuracy',
    'task_type': 'GPU',
    'verbose': 100,
    'random_seed': seed,
    'class_weights': [2, 1, 1],
    'depth': 8
})

[I 2025-07-04 01:02:32,020] A new study created in memory with name: no-name-5ad85e47-f1f8-464d-8cd7-acb21f2322e9
[I 2025-07-04 01:05:25,753] Trial 0 finished with value: 0.91829745902784 and parameters: {'bootstrap_type': 'Bayesian', 'learning_rate': 0.031126218929141865, 'l2_leaf_reg': 3.2117801620371713, 'random_strength': 9.40980091890208, 'border_count': 92, 'bagging_temperature': 0.9174011036432065}. Best is trial 0 with value: 0.91829745902784.
[I 2025-07-04 01:08:09,299] Trial 1 finished with value: 0.9515191432686435 and parameters: {'bootstrap_type': 'Bernoulli', 'learning_rate': 0.2554375200445523, 'l2_leaf_reg': 5.279459764780301, 'random_strength': 5.251811112179216, 'border_count': 66, 'subsample': 0.7434723063830708}. Best is trial 1 with value: 0.9515191432686435.
[I 2025-07-04 01:11:16,365] Trial 2 finished with value: 0.9513336404204453 and parameters: {'bootstrap_type': 'Bayesian', 'learning_rate': 0.2765678650427539, 'l2_leaf_reg': 1.9907835192618228, 'random_streng

In [ ]:
n_classes = len(np.unique(y))
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
all_test_probs = np.zeros((X_test.shape[0], n_classes))

for fold, (train_idx, valid_idx) in enumerate(kf.split(X, y)):
    print(f"🚀 Fold {fold+1} training...")

    X_train_fold, y_train_fold = X.iloc[train_idx], y.iloc[train_idx]
    X_valid_fold, y_valid_fold = X.iloc[valid_idx], y.iloc[valid_idx]

    model = CatBoostClassifier(**best_params)
    model.fit(X_train_fold, y_train_fold, cat_features=cat_features)

    fold_probs = model.predict_proba(X_test)
    all_test_probs += fold_probs

    # 변수 중요도 파악을 위한 수집
    importances = model.get_feature_importance()
    importances_list.append(importances)

# 변수 중요도 파악
mean_importances = np.mean(importances_list, axis=0)
feature_names = X.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': mean_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# 시각화
plt.figure(figsize=(10, 8))
sns.barplot(data=importance_df.head(20), x='Importance', y='Feature', palette='magma')
plt.title("Average Feature Importances (Across 10 Folds)")
plt.tight_layout()
plt.show()

avg_test_probs = all_test_probs / kf.get_n_splits()
prob_df = pd.DataFrame(avg_test_probs, columns=range(n_classes))
prob_df['ID'] = test['ID'].values

mean_probs = prob_df.groupby('ID').mean().reset_index()
mean_probs['Segment'] = mean_probs.drop(columns='ID').values.argmax(axis=1)

segment_mapping = {0: 'C', 1: 'D', 2: 'E'}
mean_probs['Segment'] = mean_probs['Segment'].map(segment_mapping)

submission = pd.DataFrame({'ID': mean_probs['ID'], 'Segment': mean_probs['Segment']})
submission.to_csv('base_catboost_kfold.csv', index=False)

print("예측 완료")

# target data(Segment) A 모델링

In [ ]:
train_A = train[train['Segment'] == 'A']

cols_to_check = [col for col in train.columns if col not in ['ID', 'Segment']]

def is_fixed_column(df, col):
    return df[col].nunique() == 1

fixed_columns_A = {col: train_A[col].iloc[0] for col in cols_to_check if is_fixed_column(train_A, col)}
max_column_values = fixed_columns_A.copy()

fixed_cols = list(max_column_values.keys())

print(f"📦 고정된 칼럼 {len(fixed_cols)}개 제거할 예정입니다.")

📦 고정된 칼럼 85개 제거할 예정입니다.


In [ ]:
matching_ids_train = train.copy()
for col, value in max_column_values.items():
    matching_ids_train = matching_ids_train[matching_ids_train[col] == value]

matching_ids_train_grouped = matching_ids_train.groupby('ID').filter(lambda x: len(x) == 6)
matching_ids_train_list = matching_ids_train_grouped['ID'].unique()

matching_ids_test = test.copy()
for col, value in max_column_values.items():
    matching_ids_test = matching_ids_test[matching_ids_test[col] == value]

matching_ids_test_grouped = matching_ids_test.groupby('ID').filter(lambda x: len(x) == 6)
matching_ids_test_list = matching_ids_test_grouped['ID'].unique()

train_filtered = train[train['ID'].isin(matching_ids_train_list)].drop(columns=fixed_cols)
test_filtered = test[test['ID'].isin(matching_ids_test_list)].drop(columns=fixed_cols)

print(f"🚀 최종 train 데이터 shape: {train_filtered.shape}")
print(f"🚀 최종 test 데이터 shape: {test_filtered.shape}")

🚀 최종 train 데이터 shape: (249594, 660)
🚀 최종 test 데이터 shape: (62586, 659)


In [ ]:
label_encoder = LabelEncoder()
train_filtered['Segment'] = label_encoder.fit_transform(train_filtered['Segment'])

X = train_filtered.drop(columns=['Segment', 'ID'])
y = train_filtered['Segment']
X_test = test_filtered.drop(columns=['ID'])

cat_features = [col for col in X.columns if X[col].dtype == 'object']
for col in cat_features:
    X[col] = X[col].astype(str)
    X_test[col] = X_test[col].astype(str)

params = {
    'iterations': 2000,
    'learning_rate': 0.05,
    'depth': 6,
    'loss_function': 'MultiClass',
    'eval_metric': 'Accuracy', # 기존엔 'MultiClass'였음.
    'verbose': 100,
    'random_seed': seed,
    'task_type': 'GPU',
    'class_weights': [20, 50, 2, 1, 1],
}

n_classes = 5
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

print(f"\n🚀 단일 Model Run 시작")

all_test_probs = np.zeros((X_test.shape[0], n_classes))

for fold, (train_idx, valid_idx) in enumerate(kf.split(X, y)):
    print(f"📂 Fold {fold + 1}")

    X_train_fold, X_valid_fold = X.iloc[train_idx], X.iloc[valid_idx]
    y_train_fold, y_valid_fold = y.iloc[train_idx], y.iloc[valid_idx]

    model = CatBoostClassifier(**params)
    model.fit(
        X_train_fold, y_train_fold,
        eval_set=(X_valid_fold, y_valid_fold),
        cat_features=cat_features,
        early_stopping_rounds=100,
        use_best_model=True
    )

    test_probs = model.predict_proba(X_test)
    all_test_probs += test_probs

    # 변수 중요도 파악을 위한 수집
    importances = model.get_feature_importance()
    importances_list.append(importances)

# 변수 중요도 파악
mean_importances = np.mean(importances_list, axis=0)
feature_names = X.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': mean_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# 시각화
plt.figure(figsize=(10, 8))
sns.barplot(data=importance_df.head(20), x='Importance', y='Feature', palette='magma')
plt.title("Average Feature Importances (Across 10 Folds)")
plt.tight_layout()
plt.show()

avg_test_probs = all_test_probs / kf.get_n_splits()
prob_df = pd.DataFrame(avg_test_probs, columns=[0, 1, 2, 3, 4])
prob_df['ID'] = test['ID'].values

mean_probs = prob_df.groupby('ID').mean().reset_index()
mean_probs['Segment'] = mean_probs[[0, 1, 2, 3, 4]].idxmax(axis=1)

segment_mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E'}
mean_probs['Segment'] = mean_probs['Segment'].map(segment_mapping)

a_ids = mean_probs.loc[mean_probs['Segment'] == 'A', 'ID'].tolist()

print(f"\n✅ A로 분류된 ID 수 = {len(a_ids)}개")
print(f"🔎 A ID: {a_ids[:50]}")


🚀 단일 Model Run 시작
📂 Fold 1
0:	learn: 0.6375698	test: 0.6371470	best: 0.6371470 (0)	total: 25.8ms	remaining: 51.5s
100:	learn: 0.7270879	test: 0.7187509	best: 0.7193422 (98)	total: 1.4s	remaining: 26.3s
200:	learn: 0.7558886	test: 0.7459522	best: 0.7462901 (198)	total: 2.76s	remaining: 24.7s
300:	learn: 0.7713088	test: 0.7585110	best: 0.7587926 (298)	total: 4.16s	remaining: 23.5s
400:	learn: 0.7829217	test: 0.7690705	best: 0.7690705 (397)	total: 5.52s	remaining: 22s
500:	learn: 0.7922539	test: 0.7776870	best: 0.7777715 (499)	total: 6.9s	remaining: 20.6s
600:	learn: 0.8003347	test: 0.7855433	best: 0.7855433 (600)	total: 8.27s	remaining: 19.3s
700:	learn: 0.8071642	test: 0.7905274	best: 0.7907527 (699)	total: 9.64s	remaining: 17.9s
800:	learn: 0.8136088	test: 0.7954552	best: 0.7960184 (799)	total: 11s	remaining: 16.5s
900:	learn: 0.8191932	test: 0.8014248	best: 0.8014248 (900)	total: 12.4s	remaining: 15.1s
1000:	learn: 0.8246430	test: 0.8062118	best: 0.8066623 (992)	total: 13.7s	remainin

# target data(Segment) B 모델링


In [ ]:
train_B = train[train['Segment'] == 'B']

cols_to_check = [col for col in train.columns if col not in ['ID', 'Segment']]

def is_fixed_column(df, col):
    return df[col].nunique() == 1

fixed_columns_B = {col: train_B[col].iloc[0] for col in cols_to_check if is_fixed_column(train_B, col)}
max_column_values = fixed_columns_B.copy()

fixed_cols = list(max_column_values.keys())

print(f"📦 고정된 칼럼 {len(fixed_cols)}개 제거할 예정입니다.")

📦 고정된 칼럼 0개 제거할 예정입니다.


In [ ]:
matching_ids_train = train.copy()
for col, value in max_column_values.items():
    matching_ids_train = matching_ids_train[matching_ids_train[col] == value]

matching_ids_train_grouped = matching_ids_train.groupby('ID').filter(lambda x: len(x) == 6)
matching_ids_train_list = matching_ids_train_grouped['ID'].unique()

matching_ids_test = test.copy()
for col, value in max_column_values.items():
    matching_ids_test = matching_ids_test[matching_ids_test[col] == value]

matching_ids_test_grouped = matching_ids_test.groupby('ID').filter(lambda x: len(x) == 6)
matching_ids_test_list = matching_ids_test_grouped['ID'].unique()

train_filtered = train[train['ID'].isin(matching_ids_train_list)].drop(columns=fixed_cols)
test_filtered = test[test['ID'].isin(matching_ids_test_list)].drop(columns=fixed_cols)

print(f"🚀 최종 train 데이터 shape: {train_filtered.shape}")
print(f"🚀 최종 test 데이터 shape: {test_filtered.shape}")

🚀 최종 train 데이터 shape: (249594, 660)
🚀 최종 test 데이터 shape: (62586, 659)


In [ ]:
label_encoder = LabelEncoder()
train_filtered['Segment'] = label_encoder.fit_transform(train_filtered['Segment'])

X = train_filtered.drop(columns=['Segment', 'ID'])
y = train_filtered['Segment']
X_test = test_filtered.drop(columns=['ID'])

cat_features = [col for col in X.columns if X[col].dtype == 'object']
for col in cat_features:
    X[col] = X[col].astype(str)
    X_test[col] = X_test[col].astype(str)

params = {
    'iterations': 1000,
    'learning_rate': 0.03,
    'depth': 8,
    'loss_function': 'MultiClass',
    'eval_metric': 'Accuracy',
    'verbose': 100,
    'random_seed': seed,
    'task_type': 'GPU',
    'class_weights': [10, 10, 1, 1, 1],
}

n_classes = 5
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

print(f"\n🚀 단일 Model Run 시작")

all_test_probs = np.zeros((X_test.shape[0], n_classes))

for fold, (train_idx, valid_idx) in enumerate(kf.split(X, y)):
    print(f"📂 Fold {fold + 1}")

    X_train_fold, X_valid_fold = X.iloc[train_idx], X.iloc[valid_idx]
    y_train_fold, y_valid_fold = y.iloc[train_idx], y.iloc[valid_idx]

    model = CatBoostClassifier(**params)
    model.fit(
        X_train_fold, y_train_fold,
        eval_set=(X_valid_fold, y_valid_fold),
        cat_features=cat_features,
        early_stopping_rounds=100,
        use_best_model=True
    )

    test_probs = model.predict_proba(X_test)
    all_test_probs += test_probs

    # 변수 중요도 파악을 위한 수집
    importances = model.get_feature_importance()
    importances_list.append(importances)

# 변수 중요도 파악
mean_importances = np.mean(importances_list, axis=0)
feature_names = X.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': mean_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# 시각화
plt.figure(figsize=(10, 8))
sns.barplot(data=importance_df.head(20), x='Importance', y='Feature', palette='magma')
plt.title("Average Feature Importances (Across 10 Folds)")
plt.tight_layout()
plt.show()

avg_test_probs = all_test_probs / kf.get_n_splits()
prob_df = pd.DataFrame(avg_test_probs, columns=[0, 1, 2, 3, 4])
prob_df['ID'] = test['ID'].values

mean_probs = prob_df.groupby('ID').mean().reset_index()
mean_probs['Segment'] = mean_probs[[0, 1, 2, 3, 4]].idxmax(axis=1)

segment_mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E'}
mean_probs['Segment'] = mean_probs['Segment'].map(segment_mapping)

b_ids = mean_probs.loc[mean_probs['Segment'] == 'B', 'ID'].tolist()

print(f"\n✅ B로 분류된 ID 수 = {len(b_ids)}개")
print(f"🔎 B ID: {b_ids[:5]}")


🚀 단일 Model Run 시작
📂 Fold 1
0:	learn: 0.6403313	test: 0.6403693	best: 0.6403693 (0)	total: 34.1ms	remaining: 34s
100:	learn: 0.7104909	test: 0.7047860	best: 0.7047860 (100)	total: 2.71s	remaining: 24.1s
200:	learn: 0.7352619	test: 0.7276845	best: 0.7276845 (200)	total: 5.28s	remaining: 21s
300:	learn: 0.7522417	test: 0.7440682	best: 0.7440682 (300)	total: 7.92s	remaining: 18.4s
400:	learn: 0.7658679	test: 0.7539369	best: 0.7539369 (400)	total: 10.5s	remaining: 15.7s
500:	learn: 0.7760202	test: 0.7623600	best: 0.7623986 (499)	total: 13.1s	remaining: 13.1s
600:	learn: 0.7851461	test: 0.7711301	best: 0.7711301 (600)	total: 15.7s	remaining: 10.5s
700:	learn: 0.7933179	test: 0.7781269	best: 0.7781269 (695)	total: 18.4s	remaining: 7.83s
800:	learn: 0.8013694	test: 0.7853549	best: 0.7853549 (800)	total: 21s	remaining: 5.22s
900:	learn: 0.8083608	test: 0.7916771	best: 0.7917928 (899)	total: 23.6s	remaining: 2.6s
999:	learn: 0.8152269	test: 0.7976330	best: 0.7976330 (999)	total: 26.3s	remaining

# combine probs, generate submisson.csv

In [ ]:
base_df = pd.read_csv('base_catboost_kfold.csv')

base_df.loc[base_df['ID'].isin(a_ids), 'Segment'] = 'A'
base_df.loc[base_df['ID'].isin(b_ids), 'Segment'] = 'B'

base_df.to_csv('final_catboost.csv', index=False)

print(f"✅ Segment가 'A'로 수정된 {len(a_ids)}개 ID 반영 완료")
print(f"✅ Segment가 'B'로 수정된 {len(b_ids)}개 ID 반영 완료")
print("🎯 최종 결과 저장 완료: final_catboost.csv")

✅ Segment가 'A'로 수정된 25개 ID 반영 완료
✅ Segment가 'B'로 수정된 0개 ID 반영 완료
🎯 최종 결과 저장 완료: final_catboost.csv
